# Práca s poľami, reťazcami a súbormi

## Inicializácia

In [105]:
import os
import math
import random
from sympy import *

In [127]:
def get_unsorted(length=100):
    """
    An unsorted array of integers.
    
    Args:
        length (int): Number of elements in the returned array (*default* 100).
    
    Returns:
        list[int]: An array full of random integers ranging from -100 to 100 (100 not included).
    """
    return [random.randrange(-100, 100) for i in range(length)]

----------

## Zoraďovanie polí

- *Poľom* nazveme akúkoľvek reprezentáciu usporiadanej $n$-tice **rovnakého dátového typu**.
    - Zoraďovací predpis, samozrejme, možno definovať i pre $n$-ticu zloženú z rôznych dátových typov. Pre názornosť ale takéto prípady vynecháme. Platia na ne rovnaké algoritmy ako na homogénne $n$-tice, teda polia v našom ponímaní.
    - Pre jednoduchosť budeme pracovať s *reálnymi číslami*
        - Prirodzene definované operátory `<`, `>`, `<=`, `>=`, `==`
    
- Cieľ: Usporiadať hodnoty v poli
    - *Vzostupne*
        - Zostupné usporiadanie je založené na rovnakom princípe, iba zmysel porovnaní bude opačný
   
--------
### Bubble sort

<img src="img/sorting/bubble_sort.gif" width=800 />

- Jeden beh:
    - Okienko o dvoch susedných hodnotách $a$ a $b$
    - Pokiaľ $a > b$, nastane zmena pozícií
    - Posun okienka o jeden prvok vpravo
    
- Pri každom behu sa na správnu pozíciu dostane aspoň jedna hodnota.
    - Pre $n$-prvkové pole teda treba $n - 1$ behov
        - Pri každom z nich $n - 1$ porovnaní

- Asymptotická zložitosť algoritmu $\mathcal{O}(n^2)$
- Schopnosť detekovať, že pole je už zoradené
    - V najlepšom prípade (už zoradené pole) je zložitosť $\mathcal{O}(n)$ (Prebehne iba 1 beh)

In [107]:
a = get_unsorted()
        
def bubble_sort(a):
    n = len(a)
    sorted = False
    
    while not sorted:
        sorted = True # Initialize the sorted flag by True
        for i in range(n - 1):
            if a[i] > a[i + 1]: # If a bad order occurs
                a[i], a[i + 1] = a[i + 1], a[i] # Swap the elements
                sorted = False # Mark the array as not sorted
                
    return a

%time bubble_sort(a)
print(a)

CPU times: user 5.79 ms, sys: 0 ns, total: 5.79 ms
Wall time: 5.87 ms
[-99, -98, -97, -97, -94, -88, -88, -86, -85, -84, -82, -80, -78, -77, -71, -70, -66, -65, -65, -64, -64, -60, -59, -59, -57, -53, -51, -48, -47, -44, -42, -39, -34, -28, -26, -24, -24, -19, -17, -16, -13, -12, -12, -11, -10, -9, -7, -5, -3, -2, -1, 2, 4, 9, 10, 14, 19, 20, 22, 25, 26, 27, 27, 28, 37, 39, 39, 39, 40, 42, 45, 45, 47, 47, 48, 51, 53, 53, 58, 60, 61, 65, 69, 72, 72, 72, 78, 79, 84, 84, 84, 84, 86, 86, 86, 87, 88, 88, 90, 97]


-----------

### Insertion sort

<img src="img/sorting/insertion_sort.gif" width=800 />

- Prvý prvok poľa je sám o sebe zoradeným poľom.
- Zoberieme 2. prvok a zaradíme ho do poľa vľavo
    - Tým získame dvojprvkové zoradené pole na začiatku celého poľa
- Zoberieme 3. prvok a zaradíme ho do dvojprvkového zoradeného poľa
    - Tým získame trojprvkové zoradené pole na začiatku celého poľa
- ...a pokračujeme až po posledný prvok celkového poľa.


- Prechádzame $n-1$ prvkov
    - Pre prvý 1 porovnanie
    - Pre druhý najviac 2 porovnania
    - Pre tretí najviac 3 porovnania atď.
- V najhoršom prípade teda celkovo $\sum_{k=1}^{n-1}k$ porovnaní

In [108]:
k, n = symbols('k n')
Sum(k, (k, 1, k - 1)).doit()

k**2/2 - k/2

- Asymptotická zložitosť $\mathcal{O}(n^2)$

In [109]:
a = get_unsorted()

def insertion_sort(a):
    n = len(a)
    for i in range(1, n):
        a[:i + 1] = bubble_sort(a[:i] + [a[i]])
        
%time insertion_sort(a)
print(a)

CPU times: user 45.9 ms, sys: 0 ns, total: 45.9 ms
Wall time: 45.7 ms
[-100, -100, -98, -95, -94, -94, -93, -93, -92, -91, -91, -91, -90, -87, -87, -81, -81, -81, -78, -77, -75, -75, -74, -73, -71, -70, -67, -63, -62, -60, -59, -58, -54, -49, -47, -47, -46, -46, -41, -39, -38, -37, -28, -27, -24, -24, -20, -17, -15, -13, -12, -10, -10, -9, -7, -4, -3, 1, 1, 1, 4, 4, 8, 12, 13, 18, 18, 19, 20, 21, 22, 27, 31, 32, 36, 38, 50, 54, 56, 59, 60, 61, 63, 64, 64, 66, 69, 75, 75, 80, 81, 82, 85, 86, 92, 93, 94, 96, 97, 99]


-----------

### Quicksort

- Patrí do skupiny algoritmov typu *rozdeľ a panuj*.
- Zaraďuje sa do triedy zložitosti $\mathcal{O}(n\log n)$
- Využitie *rekurzie*

> #### Rekurzia
>
> - Pokiaľ nie je splnená ukončovacia podmienka, funkcia **volá samú seba**.

In [110]:
def eat_cake(n_pcs):
    print('{:d} pieces left.'.format(n_pcs))
    if n_pcs > 0:
        eat_cake(n_pcs - 1)
    else:
        print('The cake is already eaten.')
        
eat_cake(7)

7 pieces left.
6 pieces left.
5 pieces left.
4 pieces left.
3 pieces left.
2 pieces left.
1 pieces left.
0 pieces left.
The cake is already eaten.


* Algoritmus quicksort:


> * Vyber **pivot** (napr. prvý prvok poľa)
>
> * Prejdi celé pole a **preusporiadaj** ho tak, aby naľavo od pivota boli iba prvky **menšie než pivot** a napravo od neho iba prvky, ktoré sú **väčšie než pivot**.
>
> * **Zoraď pomocou *quicksort*** pole vľavo od pivota i pole vpravo od pivota.

In [111]:
def quick_sort(a):
    pivot = a[0] # Set the first element as pivot
    smaller = [] # List for elements smaller than pivot
    greater = [] # List for elements greater than pivot
        
    for element in a[1:]: # For all the elements but pivot:
        if element < pivot: # If an element is smaller than pivot...
            smaller.append(element) # ...append it to the 'smaller' list
        else: # If the element is greater instead...
            greater.append(element) # ...append it to the 'greater' list
        
    smaller = quick_sort(smaller) if len(smaller) else smaller # If there is something in the 'smaller' list, call quick_sort on it
    greater = quick_sort(greater) if len(greater) else greater # If there is something in the 'greater' list, call quick_sort on it
    
    return smaller + [pivot] + greater # Build a return value as a linkage of [smaller], [pivot], and [greater]
    
    
a = get_unsorted()

%time a = quick_sort(a)
print(a)

CPU times: user 802 µs, sys: 148 µs, total: 950 µs
Wall time: 1.12 ms
[-99, -96, -95, -95, -94, -93, -90, -89, -88, -88, -87, -83, -82, -81, -78, -77, -76, -69, -69, -66, -65, -63, -61, -57, -56, -54, -51, -46, -45, -44, -44, -42, -38, -28, -24, -23, -21, -20, -20, -18, -17, -15, -14, -13, -10, -9, -2, 0, 1, 1, 3, 9, 10, 13, 14, 14, 18, 18, 25, 25, 26, 27, 33, 42, 42, 44, 47, 47, 47, 47, 47, 48, 49, 52, 53, 55, 56, 58, 59, 59, 61, 69, 69, 69, 70, 72, 74, 76, 76, 79, 86, 87, 91, 93, 94, 95, 98, 98, 99, 99]


-----------
### Bogosort

In [112]:
def issorted(a):
    return all([a[i] <= a[i + 1] for i in range(len(a) - 1)]) 

def bogo_sort(a):
    while not issorted(a):
        random.shuffle(a)
    return a

a = get_unsorted(5)
%time bogo_sort(a)

print(a)

CPU times: user 312 µs, sys: 0 ns, total: 312 µs
Wall time: 322 µs
[-84, -59, 13, 28, 61]


------------
* Zoraďovacích algoritmov existuje ešte omnoho viac $-$ viď napr. [Wikipedia](https://en.wikipedia.org/wiki/Sorting_algorithm)

### Zabudované zoraďovacie funkcie Pythonu

* `list.sort()` ([Dokumentácia](https://docs.python.org/3/library/stdtypes.html#list.sort))$~\to ~$ Zoradenie zoznamu *na mieste*

    * Parameter `reverse` (bool): `list.sort()` defaultne[<sup>1</sup>](#foot01) zoraďuje zoznam *vzostupne*. Pokiaľ je hodnota argumentu pre parameter `reverse` nastavená na  `True`, zmysel porovnávaní bude zmenený a pole bude zoradené v *zostupnom* zmysle (*default* `False`)
    
    * Parameter `key` (callable): Funkcia, ktorá transformuje hodnoty zoznamu na kľúče, ktoré majú vstúpovať do porovnávaní (*default* `None`)
        * Príklad:

In [113]:
l = [7, 'foo', 'Whale', 'Foo', 'whale', '7', 'seven', [1, 2, 3]]
# l.sort() # Error
l.sort(key=str)
print(l)

l = [7, 'foo', 'Whale', 'Foo', 'whale', '7', 'seven', [1, 2, 3]]
l.sort(key=lambda x: str(x).lower())
print(l)

[7, '7', 'Foo', 'Whale', [1, 2, 3], 'foo', 'seven', 'whale']
[7, '7', [1, 2, 3], 'foo', 'Foo', 'seven', 'Whale', 'whale']


* `sorted()` [Dokumentácia](https://docs.python.org/3/library/functions.html#sorted) $~\to ~$ Vytvorenie zoradeného poľa z iterovateľného objektu, ktorý je jej povinným parametrom
    * Parametre `key` a `reverse` majú rovnaký význam, ako v metóde `list.sort()`

----------
## Úlohy z [CW](https://cw.fel.cvut.cz/wiki/courses/bab37zpr/tutorials/lab05)

1. Napíšte funkciu `my_find(a, b)`, ktorá v reťazci `a` hľadá reťazec `b` (nepoužívajte zabudovanú funkciu `str.find()`)
    * Pokiaľ je reťazec nájdený, funkcia vráti index jeho prvého výskytu *zľava*.
    * Pokiaľ reťazec nenájde, vráti -1.
    * Efektívna implementácia: [Boyer-Moore-Horspool algoritmus](https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore%E2%80%93Horspool_algorithm).

In [114]:
# The Python find(a, b) function:
a = 'mechanotherapy'
b = 'another'

print(a.find(b))
print(a.find('that'))

4
-1


In [115]:
def my_find(a, b):
    if len(a) > len(b):
        for start in range(len(a) - len(b)):
            if a[start:start + len(b)] in b:
                return start # No need for break keyword
        return -1
    else:
        return -1
    
a = 'mechanotherapy'
b = 'another'

print(my_find(a, b))
print(my_find(a, 'that'))

4
-1


2. Napíšte funkciu `my_replace(a, b, c)`, ktorá v reťazci `a` nahradí všetky výskyty reťazca `b` reťazcom `c`. 

In [116]:
# The Python find(a, b) function:
a = 'mechanotherapy'
b = 'mechano'
c = 'hydro'

print(a.replace(b, c))

hydrotherapy


In [117]:
def my_replace(a, b, c):
    start = 0
    out = a
    while not start == -1:
        start = my_find(out, b)
        if not start == -1:
            out = out[:start] + c + out[start + len(b):]
    
    return out
    
a = 'mechanotherapy mechanotherapy'
b = 'mechano'
c = 'hydro'

print(my_replace(a, b, c))

# a = 'mechanotherapy'
# b = 'physio'
# c = 'hydro'

# print(my_replace(a, b, c))

hydrotherapy hydrotherapy


3. Napíšte program, ktorý načíta štandardný vstup a v načítanom reťazci zamení slovo `Hello` za slovo `Hi`. Pokiaľ sa vo vstupnom reťazci objaví slovo `End`, program skončí.

In [118]:
while True:
    line = input()
        
    end = line.find('End') # Index of 'End'
    
    if end + 1: # Equivalent to (if end == -1). If there is an E
        line = line[:end]  
    
    while line.find('Hello') + 1: # while there is a 'Hello' substring present
        line = line.replace('Hello', 'Hi')
        
    print(line)
    
    if end + 1: # if there is an 'End' substring present
        break

abcdHelloHellHiAbcdEndHellosHelloEndHell
abcdHiHellHiAbcd


--------
## Práca so súbormi

* Súbory všeobecne môžeme rozdeliť na
    * **Binárne** $-$ postupnosť 0 a 1 
        * Na dekódovanie je potrebné poznať *protokol*
    * **Textové**
        * Tiež ide o postupnosť 0 a 1, ale podlieha protokolu pre kódovanie textu (Unicode (UTF-8), ISO 646 (ASCII),...) ([Wikipedia](https://en.wikipedia.org/wiki/Character_encoding))
        
* **Na prípone nezáleží**
    * Prípona súboru by v ideálnom prípade mala vyjadrovať kódovanie súboru, ale neexistuje mechanizmus, ktorý by to zaručil.
        * Nie je problém uložiť obrázok vo formáte JPEG s ako súbor s príponou `.txt`, `.exe`,...
    * Názov súboru vôbec *nemusí obsahovať príponu*
        
-----
        
### Otvorenie súboru

* Funkcia `open()` ([Dokumentácia](https://docs.python.org/3/library/functions.html#open))
    * Jediný povinný parameter: *cesta k súboru*
    
#### Ako zadať cestu k súboru?

* Manuálne ako *absolútnu* cestu
    * Windows: `C:\Windows\Users\Jozko\Documents\subor.dat`
    * Unix: `/home/Jozko/Documents/subor.dat`
    
* Manuálne ako *relatívnu* cestu (volanie zo zložky `Jozko`)
    * Windows: `Documents\subor.dat`
    * Unix: `./Documents/subor.dat`
    
* S využitím balíčka `os`

>```python
>import os
>
>path2 = os.path.abspath('./Documents/subor.dat') # Call from Jozko directory
>path1 = os.path.abspath(os.path.expanduser('~/Documents/subor.dat')) # Call from anywhere
>```

In [119]:
path = os.path.abspath('.') # Path to this folder
path = os.path.abspath(os.path.expanduser('~/Documents')) # Path to the Documents folder of the current user

print(path)

/home/matej/Documents


* Prečo využívať `os`?
    * Väčšie programy sú často (skoro vždy) rozdelené do súborov, tieto súbory sú pre prehľadnosť triedené do zložiek.
    * Aby sa súbory navzájom „videli,“ musia poznať relatívne cesty k sebe (alebo aspoň k zložkám, v ktorých majú ostatné súbory vyhľadávať)
    * Balíček `os` zabezpečí, že cesty k súborom budú použiteľné na akejkoľvek platforme
    --------
    
    * Dôležitým parametrom funkcie `open()` je tiež *mód otvorenia* súboru $-$ čo bude po otvorení so súborom možné vykonávať.
        * Argument má tvar textového reťazca, zloženého z príznakov:
    
| Príznak | Význam |
| ----- | ---------------------------- |
| `'r'` | Čítanie súboru (**Default**) |
| `'w'` | Písanie do *čistého* súboru |
| `'x'` | Písanie do čistého súboru, ale iba ak súbor s takým istým menom *neexistuje* |
| `'a'` | Písanie na koniec existujúceho súboru. Ak súbor neexistuje, je vytvorený. |
| `'b'` | Zápis binárneho súboru |
| `'t'` | Zápis textového súboru (**Default**) |
| `'+'` | Otvorenie na zápis i čítanie |

> Príklad: `open('file', 'rb')`

### Zatvorenie súboru

* Pri zápise informácie do súboru sú bity najprv zbierané (tzv. *buffer* fáza) a neskôr ukladané (tzv. *flush* fáza).
* Metóda `file.close()` zabezpečí, že všetky fázy zápisu sú ukončené a zahodí referenciu na súbor.

>```python
>f = open('file')
>...
>f.close()
>```

#### Klauzula `with`

* Postará sa o korektné zatvorenie súboru (netreba volať metódu `file.close()`)
* Robí kód prehľadnejším

>```python
>with open('file', 'w') as f:
>    f.write('abcd')
>```

### Čítanie zo súboru

* `file.read()`
    * Čítanie *size* znakov (bytov v `b` móde) zo súboru. Default hodnota parametra `size` je -1 s významom *celý súbor*.
* `file.readline()`
    * Generátor, generujúci riadky súboru.
* `file.readlines()`
    * Vracia zoznam, ktorého prvky sú riadky súboru.
    * Ekvivalent `list(file.readline())`
    
### Zápis do súboru

* `file.write()`
    * Argumentom je hodnota, ktorá má byť zapísaná.
    
---------
## Dokončenie úloh z CW

4. Napíšte funkciu, ktorá načíta pole čísel zo súboru a vráti najväčšiu hodnotu v poli a zároveň jej index. Pre pole nulovej dĺžky vráti index -1.


* Textový súbor obsahuje čísla (float), oddelené čiarkami (znakom `,`). Medzi číslami môžu, ale nemusia byť zlomy riadkov.
* Príklad:

>```
>,-244,166,29
>,58,22
>,-11,-97
>
>
>,-29,-102
>,20
>,-162,134
>,91
>,187
>```

In [120]:
# Loading data
data = []
with open(os.path.abspath('data.dat'), 'r') as f:
    for line in f:
        if not line.replace('\n', ''): # If the line is empty but a newline
            continue
        ln = line.split(',')
        ln = list(filter(lambda x: x != '', ln)) # Removes empty entries
        ln = list(map(float, ln)) # Converts elements to integers
        data.extend(ln) # Appends the line-list to the final list
        
        ## --- The commands above could be chained if needed: 
        # data.extend(list(map(float, list(filter(lambda x: x != '', line.split(','))))))

In [121]:
def my_max(l):
    if len(l):
        val = l[0]
        idx = 0
        for i in range(1, len(l)):
            if l[i] > val:
                val, idx = l[i], i
            
        return val, idx
    else:
        return None, -1

In [122]:
print(my_max([-8, 3]))

(3, 1)


5. Napíšte funkciu, ktorá v poli nájde druhý najväčší prvok a vráti jeho hodnotu a index. Pre pole dĺžky menej než 2 vráti funkcia index -1.

In [123]:
def second_highest(l):
    if len(l) < 2:
        return None, -1
    else:
        # Exploring the first two values, assigning them to highest element, second higest element, and their respective indices
        highest, second = l[:2] if l[0] > l[1] else (l[1], l[0])
        idx_highest, idx_second = (0, 1) if l[0] > l[1] else (1, 0)
        
        for i in range(2, len(l)):
            if l[i] > highest:
                highest, second = l[i], highest # New second is the old highest, new highest is l[i]
                idx_highest, idx_second = i, idx_highest # Indices in the same manner
            elif l[i] > second:
                second, idx_second = l[i], i # Update the second
                
        return second, idx_second

In [124]:
a = [1,2,-8,4,0,87,4,5,8,7]

print(second_highest(a))

(8, 8)


6. Napíšte funkciu, ktorá z textového súboru načíta maticu


* Formát textového súboru: stĺpce oddelené medzerami, riadky novým riadkom.

>```
>4 -8 4.2 7
>.1 7 5. 2
>1 1 1 1
>```

In [125]:
def load_matrix(filename):
    if os.path.exists(filename): # Check for path existence
        with open(filename) as f:
            lines = f.readlines()
            if all([len(line.split(' ')) - len(lines[0].split(' ')) == 0 for line in lines]): # Each line has to contain the same amount of numbers
                matrix = [line.replace('\n', '').split(' ') for line in lines]
                return matrix
            else:
                raise Exception('Invalid matrix data.')
    else:
        raise FileNotFoundError('File not found.')

In [126]:
print(load_matrix(os.path.abspath('./matrix.dat')))

[['15', '1', '4.5', '9.', '12', '15', '1', '4.5', '9.', '12', '15', '1', '4.5', '9.', '8'], ['3', '5.2', '8', '7', '0', '-1', '9.2', '8', '7', '0', '-5', '13.2', '8', '8', '0'], ['14', '-978', '4', '1', '0', '10', '-982', '4', '9', '0', '10', '-986', '4', '1', '4'], ['15', '2', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14'], ['15', '3', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14'], ['15', '3', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14'], ['15', '3', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14'], ['15', '3', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14'], ['15', '3', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14', '15', '3', '4.7', '11.', '14'], ['15', '3', '4.8', '11.', '14', '15', '3', '4.7', '11.', '14', '15', '3', '4.7', '10.', '9'], ['4', '6.3', '10', '7', '0', '1', '9.4', '8', '9', '0', '-3', '13.4',

-----
<span id="foot01"><sup>1</sup>Vždy mám pocit, že toto slovo, hoc nespisovné, vystihuje presnú podstatu toho, čo ním chce byť povedané. Narozdiel od jeho kolegov <em>štandardne</em>, <em>prednastavene</em>, <em>bežne</em></span>.